### 1. Install BeautifulSoup and then import needed library

In [15]:
pip install beautifulsoup4

SyntaxError: invalid syntax (<ipython-input-15-5eae873b3408>, line 2)

In [16]:
pip install lxml

     |████████████████████████████████| 5.8MB 28.7MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

### 2. Get reuest fom URL, Use BeautifulSoup to scrap table from html table and assign the result to a dataframe

In [2]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
df_org=df[0]
df=df_org[df_org['Borough']!='Not assigned']

### 3. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [3]:
mask=df[df['Neighbourhood']=='Not assigned']
df['Neighbourhood'] = np.where((df.Neighbourhood == 'Not assigned'),df.Borough,df.Neighbourhood)
df.head(10)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


### 4. Group the dataframe by Postcode,Borough and then do join for the third column Neighbourhood

In [8]:
df['Neighbourhood'] = df[['Postcode','Borough','Neighbourhood']].groupby(['Postcode','Borough'])['Neighbourhood'].transform(lambda x: ','.join(x))
dfg=df[['Postcode','Borough','Neighbourhood']].drop_duplicates()
dfg.head(10)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights,Lawrence Manor,Lawrence Heigh..."
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,"Rouge,Malvern,Rouge,Malvern,Rouge,Malvern,Roug..."
13,M3B,North York,Don Mills North
14,M4B,East York,"Woodbine Gardens,Parkview Hill,Woodbine Garden..."
16,M5B,Downtown Toronto,"Ryerson,Garden District,Ryerson,Garden Distric..."


### 5. Print the shape of Final Dataframe

In [9]:
print(dfg.shape)

(103, 3)


### 6. Get the latitude and the longitude coordinates of each neighborhood.

In [4]:
#pip install geocoder
import geocoder # import geocoder
postal_code='M5G'
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

KeyboardInterrupt: 

In [11]:
df_coord=pd.read_csv('Geospatial_Coordinates.csv')
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
merged_inner = pd.merge(left=dfg,right=df_coord, left_on='Postcode', right_on='Postal Code')
merged_inner.drop("Postal Code",axis=1,inplace=True)
merged_inner.shape
merged_inner.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor,Lawrence Heigh...",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M9A,Queen's Park,Queen's Park,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern,Rouge,Malvern,Rouge,Malvern,Roug...",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill,Woodbine Garden...",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District,Ryerson,Garden Distric...",43.657162,-79.378937
